# 从word2vec到song2vec
word2vec就是将词映射到词嵌入矩阵，词与词之间的相似性可以用余弦相似度表示。

我们将歌曲的id序列取出来，类比于分完词后的句子，送到word2vec中学习一下。

原始格式

    漫步西欧小镇上##小语种,旅行##69413685##474	18682332::Wäg vo dir::Joy Amelie::70.0	4335372::Only When I Sleep::The Corrs::60.0	2925502::Si Seulement::Lynnsha::100.0	21014930::Tu N'As Pas Cherché...::La Grande Sophie::100.0	20932638::Du behöver aldrig mer vara rädd::Lasse Lindh::25.0	17100518::Silent Machine::Cat Power::60.0	3308096::Kor pai kon diew : ชอไปคนเดียว::Palmy::5.0	1648250::les choristes::Petits Chanteurs De Saint Marc::100.0	4376212::Paddy's Green Shamrock Shore::The High Kings::25.0	2925400::A Todo Color::Las Escarlatinas::95.0	19711402::Comme Toi::Vox Angeli::75.0	3977526::Stay::Blue Cafe::100.0	2538518::Shake::Elize::85.0	2866799::Mon Ange::Jena Lee::85.0	5191949::Je M'appelle Helene::Hélène Rolles::85.0	20036323::Ich Lieb' Dich Immer Noch So Sehr::Kate & Ben::100.0

In [3]:
import sys
import pickle
import gensim
import multiprocessing
from random import shuffle

In [4]:
def parse_playlist_get_sequence(in_line, playlist_sequence):
    """ 解析文本 """
    song_sequence = []
    contents = in_line.strip().split('\t')
    # 解析歌单序列
    for song in contents[1:]:
        #print('song:', song)
        try:
            song_id, song_name, artist, popularity = song.split('::')
            #print(song_id)
            song_sequence.append(song_id)
        except:
            print('song format error')
            print(song+'\n')
    shuffle(song_sequence)
    playlist_sequence.append(song_sequence)

def get_playlist_sequence(in_file, out_file):
    """ 将歌曲id传入word2vec中训练 """
    # 所有歌单序列
    playlist_sequence = []
    # 遍历所有的歌单
    for line in open(in_file):
        parse_playlist_get_sequence(line, playlist_sequence)
    # 使用word2vec进行训练
    return playlist_sequence

In [8]:
song_sequence_file = '../data/popular.playlist'
model_file = '../data/song2vec.model'
playlist_sequence = get_playlist_sequence(song_sequence_file, model_file)

song format error
1870957::彩云国物语 セカンドシリーズ::君を想う::梁邦彦::80.0

song format error
4965888::桃华月惮::龙皇-リュウオウ-::多田彰文::25.0

song format error
456177::true tears::一阵の风::菊地創::95.0

song format error
22642373::

song format error
 FAIRY TAIL メインテーマ -Slow ver.-::高梨康治::95.0

song format error
31563610::

song format error
苍之礼赞::花之祭P::60.0

song format error
4954593::リズム天国全曲集::恋の実験室::V.A.::55.0

song format error
4954596::リズム天国全曲集::シンクロ::V.A.::60.0

song format error
31654811::

song format error
American Cowboys::Tim Wynn::65.0

song format error
19169096::

song format error
 Time to Say Goodbye (Con te partirò)::Sarah Brightman::100.0

song format error
31563610::

song format error
苍之礼赞::花之祭P::60.0

song format error
31563610::

song format error
苍之礼赞::花之祭P::60.0

song format error
31563610::

song format error
苍之礼赞::花之祭P::60.0

song format error
19169096::

song format error
 Time to Say Goodbye (Con te partirò)::Sarah Brightman::100.0

song format error
376653::野弧禅狂叱(宿香之战)

song format error
:

In [7]:
#playlist_sequence[0:5]

In [9]:
len(playlist_sequence)

3772

In [10]:
def train_song2vec(playlist_sequence, model_file):
    # 使用word2vec训练
    cores = multiprocessing.cpu_count()
    print('using all' + str(cores) + 'cores')
    print('正在训练word2vec模型')
    model = gensim.models.Word2Vec(sentences=playlist_sequence, size=150, min_count=3, window=7, workers=cores)
    print('保存模型...')
    model.save(model_file)
train_song2vec(playlist_sequence, model_file)

using all4cores
正在训练word2vec模型
保存模型...


模型训练完了，让我们看看模型的训练效果吧。

In [11]:
song_id_name_set = pickle.load(open('../data/song_id_name_set.data', 'rb'))
model = gensim.models.Word2Vec.load('../data/song2vec.model')

In [12]:
song_ids = list(song_id_name_set.keys())[1300:1360]
for song_id in song_ids:
    similarily_song_list = model.most_similar(song_id)
    print(song_id, song_id_name_set[song_id], '\t|', song_id_name_set[similarily_song_list[0][0]])
    print('\n')

29567096 憾长情 (纯歌版)	绯村柯北 	| 九月	李志


34999055 纨绔	慕寒 	| 世末歌者	乐正绫


35566473 墨字吟·情（慕寒&慕斯の小乖）	慕寒 	| The Day You Went Away	M2M


34998014 雪诉离歌	齐栾 	| 酒干倘卖无	苏芮


412187499 别赋	慕寒 	| 偏偏喜欢你	陈百强


404459664 狐荼	Tacke竹桑 	| 传奇	王菲


33941486 旅行的意义	慕寒 	| 难得	安来宁


34468551 虫儿飞 - 现场版	慕寒 	| 越来越不懂	蔡健雅


34690447 花落有期	慕寒 	| 我爱你	腰乐队


34380993 落霞云归	慕寒 	| 遇见	孙燕姿


32944728 倾君沧溟恨 - 纯歌版	慕寒 	| 孙夫人（言和版）	小旭PRO


32897539 画中仙	慕寒 	| 陌生人	蔡健雅


32835224 倾君沧溟恨(剧情版)	慕寒 	| 我不难过	孙燕姿


32835225 那是一片海	慕寒 	| 连公子(剧情版) - 剧情版	文子轩


32835223 何以清尘	慕寒 	| 活得潇洒	谭咏麟


32408209 不期而遇的温暖	慕寒 	| 鹰旗	smile_小千


32807019 盛世回首	慕寒 	| 夜空中最亮的星	逃跑计划


28411504 温柔宇宙	慕寒 	| 弈剑听雨阁	小曲儿


29450491 青梅	慕寒 	| 最初的梦想	金玟岐


27928580 非鱼(少林&纯阳)	五色石南叶 	| 演员	薛之谦


28568215 樱散零乱	慕寒 	| 说好的幸福呢	周杰伦


29572769 花间事	慕寒 	| 演员	薛之谦


29393563 西窗	慕寒 	| 陪你度过漫长岁月	陈奕迅


27908610 临水照花	慕寒 	| 可惜我是水瓶座	杨千嬅


31352182  山水画意	慕寒 	| 他一定很爱你	阿杜


29418980 若与君老	慕寒 	| 至少还有你	林忆莲


29414767 封白	慕寒 	| 征服	那英


29401512 息兮	慕寒 	| 贝加尔湖畔	李健


315587 烟火莲灯	玄觞 	| 一直很安静	阿桑


29829845 梦枕红袖	慕寒 	| 今夜你会不会

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


我们可以找到和当前歌曲最相似的歌曲。